In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import scienceplots
import pandas as pd
from itertools import combinations
import pprint
from shapely.geometry import Polygon
plt.style.use(['science', 'grid', 'ieee'])

from utils import (
    load_dataset,
    get_images,
    transform_points,
    get_densities,
    predict,
    calculate_errors,
    get_result_dict,
    SCALING_FACTOR,
    plot_setup, 
    plot_setup_noised,
    plot_predictions,
    add_outlier
)

We will use two points that are similar distance to ref pt, etc. and use four points to determine a homography to have full control. Let us use points **w6, w8, w9, w22** in **DJI_0026** and validation points **b8** and **b21**. Validation point b8 has pixel density of **182.68** and an error of **xx px** and validation point b21 **346.01** and an error of **xx px**, respectively.

Now let us add random noise 100 times to the reference points and recalculate the errors. The hypothesis is, that b21 can deal better with the simulated measurement errors, as it has more pixel density. 

Results:
- std: 0, b8: 2.92, b21: 3.24
- std: 5, b8: 10.69, b21: 9.32
-  std: 10, b8: 20.52, b21: 18.42
- std: 20, b8: 42.73, b21: 35.04
- std: 30, b8: 64.38, b21: 51.56
- std: 50, b8: 124.21, b21: 94.40

Let's check it with another image and two other points to validate it once more.

## Conduct Experiment

In [ ]:
def filter_points(points, keys):
    filtered = {key: points[key] for key in keys}
    return filtered

def calc_error(pt, gt, h):
    homog_pt = np.array((pt[0], pt[1], 1))
    predicted_pt = predict(homog_pt, h)
    gt_pt = np.array(gt)
    error = np.linalg.norm(predicted_pt[:2] - gt_pt)
    return error

def add_noise(points, m=0, std=1):
    zeros = np.zeros((points.shape[0], 1))
    noise = np.random.normal(m, std, points[:,:-1].shape)
    noise = np.hstack((noise, zeros))
    return points + noise

def conduct_experiment(pv_img, tv_img, used_ref_points, n_std, val_pt1, val_pt2):
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    img_pv, img_tv = get_images(pv_img, tv_img)
    # used_ref_points = ['w6', 'w8', 'w9', 'w22']
    
    reference_pts_pv = filter_points(reference_pts_pv, used_ref_points)
    reference_pts_tv = filter_points(reference_pts_tv, used_ref_points)
        
    # Transform Points to Homogeneous Numpy arrays
    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)

    reference_pts_pv_arr_noised = add_noise(reference_pts_pv_arr, 0, n_std)
    reference_pts_tv_arr_noised = add_noise(reference_pts_tv_arr, 0, n_std)
    
    # Calculate Homography
    h, _ = cv2.findHomography(
        reference_pts_pv_arr_noised,
        reference_pts_tv_arr_noised,
        # method = cv2.RANSAC,
        method = 0,
    )
    h_inv = np.linalg.inv(h)
    error1 = calc_error(validation_pts_pv[val_pt1], validation_pts_tv[val_pt1], h)
    error2 = calc_error(validation_pts_pv[val_pt2], validation_pts_tv[val_pt2], h)
    return error1, error2

In [ ]:
used_ref_pts = ['w6', 'w8', 'w9', 'w22']
used_val_pts = ['b8', 'b21']
used_ref_pts2 = ['w21', 'w23', 'w28', 'w26']
used_val_pts2 = ['b28', 'b24']

def get_results(pv_img, tv_img, used_ref_pts, used_val_pts):
    results = {}
    for std in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]:
        results[std] = {}
        results[std][used_val_pts[0]] = []
        results[std][used_val_pts[1]] = []
        errors = []
        for i in range(100):
            _errors = conduct_experiment(pv_img, tv_img, used_ref_pts, std, used_val_pts[0], used_val_pts[1])
            results[std][used_val_pts[0]].append(_errors[0])
            results[std][used_val_pts[1]].append(_errors[1])
    return results
results1 = get_results('IMG_01', 'IMG_00', used_ref_pts, used_val_pts)
results2 = get_results('IMG_02', 'IMG_00', used_ref_pts2, used_val_pts2)

## Print Results 

In [ ]:
pv_img = 'IMG_01'
tv_img = 'IMG_00'

_errors = conduct_experiment(pv_img, tv_img, used_ref_pts, 0, used_val_pts[0], used_val_pts[1])
print(f'std: 0, {used_val_pts[0]}: {_errors[0]:.2f}, {used_val_pts[1]}: {_errors[1]:.2f}')
for std, result_d in results1.items():
    b8_error = np.mean(result_d[used_val_pts[0]])
    b21_error = np.mean(result_d[used_val_pts[1]])
    print(f'std: {std}, {used_val_pts[0]}: {b8_error:.2f}, {used_val_pts[1]}: {b21_error:.2f}')

In [ ]:
pv_img = 'IMG_02'
tv_img = 'IMG_00'
_errors = conduct_experiment('IMG_02', tv_img, used_ref_pts2, 0, used_val_pts2[0], used_val_pts2[1])
print(f'std: 0, {used_val_pts2[0]}: {_errors[0]:.2f}, {used_val_pts2[1]}: {_errors[1]:.2f}')
for std, result_d in results2.items():
    b8_error = np.mean(result_d[used_val_pts2[0]])
    b21_error = np.mean(result_d[used_val_pts2[1]])
    print(f'std: {std}, {used_val_pts2[0]}: {b8_error:.2f}, {used_val_pts2[1]}: {b21_error:.2f}')

## Plot Setup

In [ ]:
def plot_reference_point(name, x, y, ax, fc='white', ec='black', x_offset=-50, y_offset=0, fontcolor='black'):
    ax.plot(x, y, marker='X', markersize=8, color='whitesmoke')
    ax.annotate(name, 
        xy=(x, y),
        color=fontcolor,
        bbox=dict(facecolor=fc, edgecolor=ec, boxstyle="round", alpha=0.8)
    )
    
def plot_reference_points(reference_pts_pv, ax):
    for name, pt in reference_pts_pv.items(): 
        plot_reference_point(name, pt[0], pt[1], ax)

def plot_validation_points(validation_pts_pv, ax):
    for name, pt in validation_pts_pv.items(): 
        plot_reference_point(name, pt[0], pt[1], ax, fc='blue', ec='black', fontcolor='white')

In [ ]:
pv_img = 'IMG_01'
tv_img = 'IMG_00'
used_val_points = ['b8', 'b21']
img_pv, img_tv = get_images(pv_img, tv_img)
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
reference_pts_pv = filter_points(reference_pts_pv, used_ref_pts)
validation_pts_pv = filter_points(validation_pts_pv, used_val_points)

ax.imshow(img_pv)
ax.axis('off')
plot_reference_points(reference_pts_pv, ax)
plot_validation_points(validation_pts_pv, ax)

In [ ]:
pv_img = 'IMG_02'
tv_img = 'IMG_00'
used_ref_points = ['w21', 'w23', 'w28', 'w26']
used_val_points = ['b28', 'b24']
img_pv, img_tv = get_images(pv_img, tv_img)
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
reference_pts_pv = filter_points(reference_pts_pv, used_ref_points)
validation_pts_pv = filter_points(validation_pts_pv, used_val_points)

ax.imshow(img_pv)
ax.axis('off')
plot_reference_points(reference_pts_pv, ax)
plot_validation_points(validation_pts_pv, ax)

## Plot Results

In [ ]:
fig, ax = plt.subplots(1, 1)#, figsize=(10, 10))

errors_bad = []
errors_good = []
stds = [0]

error = conduct_experiment('IMG_01', tv_img, used_ref_pts, 0, used_val_pts[0], used_val_pts[1])
errors_bad.append(error[0])
errors_good.append(error[1])

for std, result_dict in results1.items():
    stds.append(std)
    errors_bad.append(np.mean(result_dict['b8']))
    errors_good.append(np.mean(result_dict['b21']))
    
ax.plot(stds, errors_bad, color='black', marker='o', label='b8')
ax.plot(stds, errors_good, color='gray', marker='o', label='b21')
ax.set_xlabel('standard deviation [px]')
ax.set_ylabel('mean error [px]')

ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.0),
          ncol=3, fancybox=True, shadow=False, framealpha=0.8)

ax.annotate(
    'pixel density of b8: 183', 
    xy=(5, 100),
    color='black',
    fontsize=7,
    bbox=dict(facecolor='white', edgecolor='black', boxstyle="round", pad=0.5, lw=0.7)
)
ax.annotate(
    'pixel density of b21: 346', 
    xy=(30, 22),
    color='black',
    fontsize=7,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle="round", pad=0.5, lw=0.7)
)
ax.grid(False)

plt.savefig('output/pixel_density_0026.png', dpi=600)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1)#, figsize=(10, 10))

errors_bad = []
errors_good = []
stds = [0]

error = conduct_experiment('IMG_02', tv_img, used_ref_pts2, 0, used_val_pts2[0], used_val_pts2[1])
errors_bad.append(error[0])
errors_good.append(error[1])

for std, result_dict in results2.items():
    stds.append(std)
    errors_bad.append(np.mean(result_dict['b24']))
    errors_good.append(np.mean(result_dict['b28']))
    
ax.plot(stds, errors_bad, color='black', marker='o', label='b24')
ax.plot(stds, errors_good, color='gray', marker='o', label='b28')

ax.set_xlabel('standard deviation [px]')
ax.set_ylabel('mean error [px]')

ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.0),
          ncol=3, fancybox=True, shadow=False, framealpha=0.8)

ax.annotate(
    'pixel density of b24: 138', 
    xy=(5, 100),
    color='black',
    fontsize=7,
    bbox=dict(facecolor='white', edgecolor='black', boxstyle="round", pad=0.5, lw=0.7)
)
ax.annotate(
    'pixel density of b28: 358', 
    xy=(35, 22),
    color='black',
    fontsize=7,
    bbox=dict(facecolor='white', edgecolor='gray', boxstyle="round", pad=0.5, lw=0.7)
)
ax.grid(False)

plt.savefig('output/pixel_density_0029.png', dpi=600)
plt.show()